<a href="https://colab.research.google.com/github/Trading-com-Dados/tutoriais_publicos/blob/main/20220517_Stock_Screener_Python_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://tradingcomdados.com/images/logotipo/logotipo-trading-com-dados.svg" width="300" align="left"/>

# **Stock Screener com Python - modelo simplificado Twitter**

### **1. Bibliotecas utilizadas**

Para um Screening em tempo real intraday, a melhor biblioteca seria MetaTrader5 (integra o código com o software MetaTrader 5 - MT5).<p>
Como a maioria do público talvez não tenha acesso ao MT5, vamos fazer um modelo de Screening usando o Yahoo Finance, mas que tem um delay de alguns minutos e, nesse caso, não indicado para screening intraday. 

In [77]:
!pip install yfinance
!pip install ta

In [78]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import pytz
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
import ta as ta

### **2. Organizando o dataset**

#### 2.1. Baixar dados de cotações

In [79]:
ativos_IBOV = ['WEGE3', 'EMBR3', 'AZUL4', 'CCRO3', 'ECOR3', 'GOLL4', 'RAIL3', 'POSI3', 'BRFS3', 'JBSS3',
               'MRFG3', 'BEEF3', 'ABEV3', 'ASAI3', 'CRFB3', 'PCAR3', 'NTCO3', 'AMER3', 'SOMA3', 'LREN3',
               'MGLU3', 'PETZ3', 'VIIA3', 'ALPA4', 'CYRE3', 'EZTC3', 'JHSF3', 'MRVE3', 'CVCB3', 'COGN3',
               'RENT3', 'LCAM3', 'YDUQ3', 'BRML3', 'IGTI11', 'MULT3', 'BIDI11', 'BPAN4', 'BBDC3', 'BBDC4',
               'BBAS3', 'BPAC11', 'ITSA4', 'ITUB4', 'SANB11', 'BBSE3', 'IRBR3', 'SULA11', 'B3SA3', 'CIEL3',
               'DXCO3', 'KLBN11', 'SUZB3', 'BRAP4', 'CMIN3', 'VALE3', 'BRKM5', 'GGBR4', 'GOAU4', 'CSNA3',
               'USIM5', 'RRRP3', 'CSAN3', 'PETR3', 'PETR4', 'PRIO3', 'UGPA3', 'VBBR3', 'HYPE3', 'RADL3',
               'FLRY3', 'HAPV3', 'QUAL3', 'RDOR3', 'LWSA3', 'CASH3', 'TOTS3', 'VIVT3', 'TIMS3', 'SBSP3',
               'CMIG4', 'CPLE6', 'CPFE3', 'ELET3', 'ELET6', 'ENBR3', 'ENGI11', 'ENEV3', 'EGIE3', 'EQTL3',
               'TAEE11']

In [80]:
tickers_IBOV_yf = [ativos_IBOV + '.SA' for ativos_IBOV in ativos_IBOV]
tickers_IBOV_yf.sort()
tickers_IBOV_yf[0:5]

['ABEV3.SA', 'ALPA4.SA', 'AMER3.SA', 'ASAI3.SA', 'AZUL4.SA']

In [81]:
data_fim = datetime.now(pytz.timezone("America/Sao_Paulo"))
data_fim = data_fim.strftime('%Y-%m-%d')
data_fim

'2022-05-18'

In [82]:
data_inicio = datetime.now(pytz.timezone("America/Sao_Paulo")) - timedelta(days=5)
data_inicio = data_inicio.strftime('%Y-%m-%d')
data_inicio

'2022-05-13'

In [83]:
lista_OHLC = []

for i in tickers_IBOV_yf:
  cotacoes_IBOV = yf.download([i], start=data_inicio, end=data_fim, interval = "5m")
  cotacoes_IBOV['Ativo'] = i
  lista_OHLC.append(cotacoes_IBOV)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [84]:
lista_OHLC[0]

,Open,High,Low,Close,Adj Close,Volume,Ativo
Datetime,,,,,,,
2022-05-13 10:00:00-03:00,14.55,14.55,14.51,14.52,14.52,0,ABEV3.SA
2022-05-13 10:05:00-03:00,14.52,14.56,14.51,14.53,14.53,150000,ABEV3.SA
2022-05-13 10:10:00-03:00,14.53,14.54,14.45,14.52,14.52,136800,ABEV3.SA
2022-05-13 10:15:00-03:00,14.51,14.54,14.48,14.49,14.49,78000,ABEV3.SA
2022-05-13 10:20:00-03:00,14.50,14.53,14.47,14.47,14.47,70000,ABEV3.SA
...,...,...,...,...,...,...,...
2022-05-17 16:30:00-03:00,14.73,14.74,14.72,14.73,14.73,166400,ABEV3.SA
2022-05-17 16:35:00-03:00,14.73,14.75,14.72,14.72,14.72,191000,ABEV3.SA
2022-05-17 16:40:00-03:00,14.72,14.75,14.71,14.74,14.74,262800,ABEV3.SA


#### 2.2. Calcular indicadores de Análise Técnica

In [89]:
for i in range(0,len(lista_OHLC)):
  MM9p = ta.trend.SMAIndicator(lista_OHLC[i]['Close'], window=9)
  MM20p = ta.trend.SMAIndicator(lista_OHLC[i]['Close'], window=20)
  MM200p = ta.trend.SMAIndicator(lista_OHLC[i]['Close'], window=200)
  IFR14p = ta.momentum.RSIIndicator(close=lista_OHLC[i]['Close'], window=14)
  lista_OHLC[i]['SMA_9p'] = MM20p.sma_indicator()
  lista_OHLC[i]['SMA_20p'] = MM20p.sma_indicator()
  lista_OHLC[i]['SMA_200p'] = MM200p.sma_indicator()
  lista_OHLC[i]['IFR14p'] = IFR14p.rsi()

In [90]:
lista_OHLC[0]

,Open,High,Low,Close,Adj Close,Volume,Ativo,SMA_9p,SMA_20p,SMA_200p,IFR14p
Datetime,,,,,,,,,,,
2022-05-13 10:00:00-03:00,14.55,14.55,14.51,14.52,14.52,0,ABEV3.SA,NaN,NaN,NaN,NaN
2022-05-13 10:05:00-03:00,14.52,14.56,14.51,14.53,14.53,150000,ABEV3.SA,NaN,NaN,NaN,NaN
2022-05-13 10:10:00-03:00,14.53,14.54,14.45,14.52,14.52,136800,ABEV3.SA,NaN,NaN,NaN,NaN
2022-05-13 10:15:00-03:00,14.51,14.54,14.48,14.49,14.49,78000,ABEV3.SA,NaN,NaN,NaN,NaN
2022-05-13 10:20:00-03:00,14.50,14.53,14.47,14.47,14.47,70000,ABEV3.SA,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2022-05-17 16:30:00-03:00,14.73,14.74,14.72,14.73,14.73,166400,ABEV3.SA,14.7325,14.7325,14.71025,45.358220
2022-05-17 16:35:00-03:00,14.73,14.75,14.72,14.72,14.72,191000,ABEV3.SA,14.7290,14.7290,14.71100,42.994737
2022-05-17 16:40:00-03:00,14.72,14.75,14.71,14.74,14.74,262800,ABEV3.SA,14.7275,14.7275,14.71190,48.747147


## 2.3. Visualização gráficos candlesticks e indicadores

In [91]:
list(enumerate(tickers_IBOV_yf))

[(0, 'ABEV3.SA'),
 (1, 'ALPA4.SA'),
 (2, 'AMER3.SA'),
 (3, 'ASAI3.SA'),
 (4, 'AZUL4.SA'),
 (5, 'B3SA3.SA'),
 (6, 'BBAS3.SA'),
 (7, 'BBDC3.SA'),
 (8, 'BBDC4.SA'),
 (9, 'BBSE3.SA'),
 (10, 'BEEF3.SA'),
 (11, 'BIDI11.SA'),
 (12, 'BPAC11.SA'),
 (13, 'BPAN4.SA'),
 (14, 'BRAP4.SA'),
 (15, 'BRFS3.SA'),
 (16, 'BRKM5.SA'),
 (17, 'BRML3.SA'),
 (18, 'CASH3.SA'),
 (19, 'CCRO3.SA'),
 (20, 'CIEL3.SA'),
 (21, 'CMIG4.SA'),
 (22, 'CMIN3.SA'),
 (23, 'COGN3.SA'),
 (24, 'CPFE3.SA'),
 (25, 'CPLE6.SA'),
 (26, 'CRFB3.SA'),
 (27, 'CSAN3.SA'),
 (28, 'CSNA3.SA'),
 (29, 'CVCB3.SA'),
 (30, 'CYRE3.SA'),
 (31, 'DXCO3.SA'),
 (32, 'ECOR3.SA'),
 (33, 'EGIE3.SA'),
 (34, 'ELET3.SA'),
 (35, 'ELET6.SA'),
 (36, 'EMBR3.SA'),
 (37, 'ENBR3.SA'),
 (38, 'ENEV3.SA'),
 (39, 'ENGI11.SA'),
 (40, 'EQTL3.SA'),
 (41, 'EZTC3.SA'),
 (42, 'FLRY3.SA'),
 (43, 'GGBR4.SA'),
 (44, 'GOAU4.SA'),
 (45, 'GOLL4.SA'),
 (46, 'HAPV3.SA'),
 (47, 'HYPE3.SA'),
 (48, 'IGTI11.SA'),
 (49, 'IRBR3.SA'),
 (50, 'ITSA4.SA'),
 (51, 'ITUB4.SA'),
 (52, 'JBSS3.SA'),

In [92]:
# Como exemplo, vamos produzir os gráficos de candlestick (com suas MMs), OBV e IFR do ticker 'MGLU3.SA' (número 59 na lista)

df_figure = lista_OHLC[0]

fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Candlestick(name='ABEV3', x=df_figure.index, open=df_figure['Open'], high = df_figure['High'], low=df_figure['Low'], close=df_figure['Close']), row=1,col=1)
fig.add_trace(go.Scatter(name='SMA_9p', x=df_figure.index, y=df_figure['SMA_9p'],marker_color='red'), row=1, col=1)
fig.add_trace(go.Scatter(name='SMA_20p', x=df_figure.index, y=df_figure['SMA_20p'],marker_color='blue'), row=1, col=1)
fig.add_trace(go.Scatter(name='SMA_200p', x=df_figure.index, y=df_figure['SMA_200p'],marker_color='black'), row=1, col=1)

fig.add_trace(go.Scatter(name='IFR14p', x=df_figure.index, y=df_figure['IFR14p'],marker_color='orange'), row=2, col=1)

fig.update_xaxes(title_text="<b> Data",rangebreaks=[dict(bounds=["sat", "mon"]),dict(bounds=[17, 10], pattern="hour")])

fig.update_layout(xaxis_rangeslider_visible=False, title_text='Price, OBV e IFR',width=1000,height=1000)
fig.show()

# 3. Screening dos ativos listados

Vamos determinar algumas regras e fazer a varredura de quantos e quais ativos se encaixam nesse momento. Estaremos buscando ativos que apresentaram:

* Regra 01: Fechamento com preço acima da MM20p e MM200p;
* Regra 02: MM9p > MM20p > MM200p;
* Regra 03: IFR14p < 50

In [93]:
lista_OHLC[0]

,Open,High,Low,Close,Adj Close,Volume,Ativo,SMA_9p,SMA_20p,SMA_200p,IFR14p
Datetime,,,,,,,,,,,
2022-05-13 10:00:00-03:00,14.55,14.55,14.51,14.52,14.52,0,ABEV3.SA,NaN,NaN,NaN,NaN
2022-05-13 10:05:00-03:00,14.52,14.56,14.51,14.53,14.53,150000,ABEV3.SA,NaN,NaN,NaN,NaN
2022-05-13 10:10:00-03:00,14.53,14.54,14.45,14.52,14.52,136800,ABEV3.SA,NaN,NaN,NaN,NaN
2022-05-13 10:15:00-03:00,14.51,14.54,14.48,14.49,14.49,78000,ABEV3.SA,NaN,NaN,NaN,NaN
2022-05-13 10:20:00-03:00,14.50,14.53,14.47,14.47,14.47,70000,ABEV3.SA,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2022-05-17 16:30:00-03:00,14.73,14.74,14.72,14.73,14.73,166400,ABEV3.SA,14.7325,14.7325,14.71025,45.358220
2022-05-17 16:35:00-03:00,14.73,14.75,14.72,14.72,14.72,191000,ABEV3.SA,14.7290,14.7290,14.71100,42.994737
2022-05-17 16:40:00-03:00,14.72,14.75,14.71,14.74,14.74,262800,ABEV3.SA,14.7275,14.7275,14.71190,48.747147


In [94]:
ls_screening = []

for i in range(0,len(lista_OHLC)):
  if (lista_OHLC[i].iloc[-1]['Close'] >= lista_OHLC[i].iloc[-1]['SMA_20p']) and (lista_OHLC[i].iloc[-1]['Close'] >= lista_OHLC[i].iloc[-1]['SMA_200p']) and (lista_OHLC[i].iloc[-1]['SMA_9p'] >= lista_OHLC[i].iloc[-1]['SMA_20p']) and (lista_OHLC[i].iloc[-1]['SMA_20p'] >= lista_OHLC[i].iloc[-1]['SMA_200p']) and (lista_OHLC[i].iloc[-1]['IFR14p'] < 50):
    ls_screening.append((tickers_IBOV_yf[i],i))

In [95]:
ls_screening

[('CPFE3.SA', 24),
 ('DXCO3.SA', 31),
 ('ECOR3.SA', 32),
 ('KLBN11.SA', 54),
 ('LCAM3.SA', 55),
 ('RAIL3.SA', 71),
 ('VIVT3.SA', 88)]

In [96]:
ls_screening = []

for i in range(0,len(lista_OHLC)):
  if (lista_OHLC[i].iloc[-1]['Close'] >= lista_OHLC[i].iloc[-1]['SMA_20p']) and (lista_OHLC[i].iloc[-1]['Close'] >= lista_OHLC[i].iloc[-1]['SMA_200p']) and (lista_OHLC[i].iloc[-1]['SMA_9p'] >= lista_OHLC[i].iloc[-1]['SMA_20p']) and (lista_OHLC[i].iloc[-1]['SMA_20p'] >= lista_OHLC[i].iloc[-1]['SMA_200p']) and (lista_OHLC[i].iloc[-1]['IFR14p'] < 50):
    ls_screening.append((tickers_IBOV_yf[i],i))
    print("O ativo indexado como",i,"("+tickers_IBOV_yf[i]+") atende a todas as regras estabelecidas")
  else:
    print(tickers_IBOV_yf[i],"não atende")

ABEV3.SA não atende
ALPA4.SA não atende
AMER3.SA não atende
ASAI3.SA não atende
AZUL4.SA não atende
B3SA3.SA não atende
BBAS3.SA não atende
BBDC3.SA não atende
BBDC4.SA não atende
BBSE3.SA não atende
BEEF3.SA não atende
BIDI11.SA não atende
BPAC11.SA não atende
BPAN4.SA não atende
BRAP4.SA não atende
BRFS3.SA não atende
BRKM5.SA não atende
BRML3.SA não atende
CASH3.SA não atende
CCRO3.SA não atende
CIEL3.SA não atende
CMIG4.SA não atende
CMIN3.SA não atende
COGN3.SA não atende
O ativo indexado como 24 (CPFE3.SA) atende a todas as regras estabelecidas
CPLE6.SA não atende
CRFB3.SA não atende
CSAN3.SA não atende
CSNA3.SA não atende
CVCB3.SA não atende
CYRE3.SA não atende
O ativo indexado como 31 (DXCO3.SA) atende a todas as regras estabelecidas
O ativo indexado como 32 (ECOR3.SA) atende a todas as regras estabelecidas
EGIE3.SA não atende
ELET3.SA não atende
ELET6.SA não atende
EMBR3.SA não atende
ENBR3.SA não atende
ENEV3.SA não atende
ENGI11.SA não atende
EQTL3.SA não atende
EZTC3.SA não

In [97]:
# Checagem gráfica de um ativo que atende a todas as regras do Screening de Compra
# Cheque todas as regras nos gráficos e veja que realmente funcionou

df_figure = lista_OHLC[88]

fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Candlestick(name='VIVT3', x=df_figure.index, open=df_figure['Open'], high = df_figure['High'], low=df_figure['Low'], close=df_figure['Close']), row=1,col=1)
fig.add_trace(go.Scatter(name='SMA_9p', x=df_figure.index, y=df_figure['SMA_9p'],marker_color='red'), row=1, col=1)
fig.add_trace(go.Scatter(name='SMA_20p', x=df_figure.index, y=df_figure['SMA_20p'],marker_color='blue'), row=1, col=1)
fig.add_trace(go.Scatter(name='SMA_200p', x=df_figure.index, y=df_figure['SMA_200p'],marker_color='black'), row=1, col=1)

fig.add_trace(go.Scatter(name='IFR14p', x=df_figure.index, y=df_figure['IFR14p'],marker_color='orange'), row=2, col=1)

fig.update_xaxes(title_text="<b> Data",rangebreaks=[dict(bounds=["sat", "mon"]),dict(bounds=[17, 10], pattern="hour")])

fig.update_layout(xaxis_rangeslider_visible=False, title_text='Price, OBV e IFR',width=1000,height=1000)
fig.show()